In [1]:
import cv2
import torch
import random
import torchreid
import numpy as np
from PIL import Image
from time import time
from glob import glob
from threading import Thread
from datetime import datetime
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## P_RI Model

In [3]:
# datamanager = torchreid.data.ImageDataManager(
#     root="reid-data",
#     sources="market1501",
#     targets="market1501",
#     height=256,
#     width=128,
#     batch_size_train=32,
#     batch_size_test=100,
#     transforms=["random_flip", "random_crop"]
# )

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize((256, 128)) #(160, 64)
])

In [4]:
myModel = torchreid.models.build_model(
    name="xception",
    num_classes=751, #datamanager.num_train_pids
    loss="softmax",
    pretrained=True
)

In [5]:
checkpoint = torch.load('model.pth.tar-80')
# checkpoint = torch.load('./log5/HaCNN/model/model.pth.tar-120')
myModel.load_state_dict(checkpoint['state_dict'])

myModel = myModel.cuda()
myModel.eval()

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d(
        (conv1): Conv2d(128, 128, kernel_size=(3

## Yolo Model 

In [6]:
things_model = torch.hub.load('ultralytics/yolov5', 'custom', path='./yolov5x6.pt',device='cuda:0')
%matplotlib inline

Using cache found in C:\Users\a.nasimi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-2 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


# Camera Settings 

In [7]:
class ThreadedCamera(object):
    def __init__(self, source='cam:Ashkan123@192.168.172.199:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

# Handeling Functions

In [8]:
def Write_On_Image(label,cor,img_original):
    ROI_Region = [0,0,0,0]
    try: color = rgb_colors[label]
    except KeyError:
        r = random.randrange(1, 256, 20)
        g = random.randrange(1, 256, 20)
        b = random.randrange(1, 256, 20)
        color = (r,g,b)
        rgb_colors[label] = color

    img_original = cv2.rectangle(np.array(img_original), (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1],abs(cor[2]-(cor[0])),abs(cor[3]-(cor[1]))), color, 2)
    img_original = cv2.putText(np.array(img_original), f"No: {label}", (cor[0]+ROI_Region[0], cor[1]+ROI_Region[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, color,2,cv2.LINE_AA)
    
    return img_original

In [9]:
def Pre_Proc_Light(img):
    lightness = np.mean(img)
    if lightness < 95:
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
    else: return img

In [10]:
def Person_Detection(img,img_orig,ROI_Region):
    imgs = []
    save_flag = True
    img_original = img_orig.copy()
    
    now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
    img = np.array(Image.fromarray(img))
    img = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2RGB)
    
    with torch.no_grad(): results = things_model(img)

    if len(results.xyxy[0]) == 0: pass
    else:
        img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_BGR2RGB)
        
        for indx,res in enumerate(results.xyxy[0]):
            
            predict = results.pandas().xyxy[0]['name'][indx]
            
            if predict == 'person': pass
            else: continue

            res = np.array(res.detach().cpu())
            if res[4]*100 < 45: save_flag = False
            else: 
                x = round(res[0])
                y = round(res[1])
                w = round(res[2])
                h = round(res[3])
                
                imgs.append((img[y:h,x:w],[x,y,h,w]))
                
#                 img_original = cv2.rectangle(np.array(img_original), (x+ROI_Region[0], y+ROI_Region[1],abs(w-(x)),abs(h-(y))), (255,0,0), 2)
#                 img_original = cv2.putText(np.array(img_original), f"{predict}", (x+ROI_Region[0], y+ROI_Region[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)               

#         if save_flag and not (len(imgs) == 0):
#             img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_RGB2BGR)
#             cv2.imwrite(f'./Images/{now_time}.jpg',img_original)
#             img_original = cv2.cvtColor(np.array(img_original),cv2.COLOR_BGR2RGB)
#         else: pass
        
    return imgs

In [11]:
def Person_ReIdentification(img,thresh=13,flag=False,indx=None):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = Pre_Proc_Light(img)
    img_ = transform(img)
    x = myModel(img_.unsqueeze(0).float().cuda()).detach().cpu()
    
    if flag:
        
        if indx is None: indx = len(database) + 1
        else: pass
        
        database.append((indx,x))
        
#         if indx == len(database): cv2.imwrite(f'./Samples_on_Xception/Sample_4/6-enhance light - Thresh 10 - 6 views/Sample{indx}_1-light.png',img)
#         else: cv2.imwrite(f'./Samples_on_Xception/Sample_4/6-enhance light - Thresh 10 - 6 views/Sample{indx}_2-light.png',img)
        
        return indx,None,True
    
    else:
        for indx,embd in database:
            dist = torch.cdist(x, embd, p=2.0)[0][0]
            if dist < thresh: return indx,dist,True
            else: continue
        return None,None,False

In [12]:
def Person_ReIdentification(img,thresh=13,flag=False,flag_unknown=False,indx=None):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = Pre_Proc_Light(img)
    img_ = transform(img)
    x = myModel(img_.unsqueeze(0).float().cuda()).detach().cpu()
    
    if flag and not flag_unknown:
        if indx is None: indx = f'{len(database) + 1}'
        else: pass
        
        database.append((indx,x))
   
        return indx,None,True
    
    elif flag_unknown and not flag:
        if indx is None: indx = f'{len(database_unknowns) + 1}'
        else: pass
        
        database_unknowns.append((f'Unknown {indx}',x))
   
        return indx,None,True

    elif not(flag and flag_unknown):

        distns = []
        for indx,embd in database:
            dist = torch.cdist(x, embd, p=2.0)[0][0]
            distns.append((indx,dist))
            
        try: indx,dist = min(distns, key=lambda x: x[1])
        except ValueError: return None,None,False
        
        if dist < thresh: return indx,dist,True
        
        else: 
            for indx,embd in database_unknowns:
                dist = torch.cdist(x, embd, p=2.0)[0][0]
                distns.append((indx,dist))

            indx,dist = min(distns, key=lambda x: x[1])
            
            if dist < thresh: return indx,dist,True 
            elif dist < (thresh + 2): 
                database_unknowns.append((indx,x))
                return indx,dist,True 
            else: return None,None,False
    else: return None,None,False

In [13]:
def Merge_Person_ReIdentification(thresh=13):
    removed = []
    global database_unknowns
    
    for indx_unknown,emb_unknown in database_unknowns:
        distns = []
        for indx,embd in database:
            dist = torch.cdist(emb_unknown, embd, p=2.0)[0][0]
            distns.append((indx,dist,emb_unknown))

        indx,dist,x = min(distns, key=lambda x: x[1])
        
        if dist < (thresh + 2): 
            database.append((indx,x))
            removed.append((indx_unknown,emb_unknown))
        else: continue
    
    database_unknowns1 = []
    for ele,embds in database_unknowns:
        for ele1,embds1 in removed:
            if ele  == ele1 and embds.all() == embds1.all(): continue
            else: database_unknowns1.append((ele,embds)) 
    
    database_unknowns = database_unknowns1

In [14]:
# from pathlib import Path
# import tensorflow as tf
# from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

# bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))

# def Blurring(image):

# #     image_array = tf.keras.preprocessing.image.img_to_array(image)
#     result = bodypix_model.predict_single(image)

#     # simple mask
#     mask = result.get_mask(threshold=0.20)
#     res = cv2.bitwise_and(image,image,mask = np.uint8(mask))
    
#     blur = cv2.blur(image,(45,45),0)
#     out = image.copy()

#     sharpen_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
#     sharpen = cv2.filter2D(out, -1, sharpen_kernel)

#     sharpen[res==0] = blur[res==0]
 
#     return sharpen
    

# #     # colored mask (separate colour for each body part)
# #     colored_mask = result.get_colored_part_mask(mask)

# Evaluate

In [15]:
def mouse_click(event, x, y, flags, param):
    global img1
    global imgs
    global database
    global ROI_Region
    global ROI_Person
    global img_original

    if event == cv2.EVENT_LBUTTONDOWN: 
        if len(database) == 0: pass
        else: database = []
        print("Hei2")    
        ROI_Person = cv2.selectROI("Select Area",img_original)
        print(ROI_Person)
        img1 = img_original[int(ROI_Person[1]):int(ROI_Person[1]+ROI_Person[3]),int(ROI_Person[0]):int(ROI_Person[0]+ROI_Person[2])]
        print(img1)
        imgs = Person_Detection(img1,img1_original,ROI_Region)
        print(img)
        if len(imgs) == 1: 
            _,flag = Person_ReIdentification(imgs[0],True)
            
            if flag:
                print('Person already exist')
                ROI_Person = [0,0,0,0]
            else: pass
            
        else:
            print('no Person detected')
            ROI_Person = [0,0,0,0]
            
#         cv2.destroyAllWindows()
    elif event == cv2.EVENT_RBUTTONDOWN:
        ROI_Region = cv2.selectROI("Select Area",img_original)
        cv2.destroyAllWindows()
    elif event == cv2.EVENT_MBUTTONDOWN: ROI_Region = [0,0,0,0]
    else: pass

# Evaluate On Recorded Video File

In [16]:
ROI_Region,ROI_Person = [0,0,0,0],[0,0,0,0]
img_original,database,database_unknowns,rgb_colors = None,[],[],{}

In [17]:
counter = 0
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_6/4/Sample4*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
    img = Blurring(imgs[0][0])
#     img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Person_ReIdentification(img,flag=True,indx=f'Ezati')
#         indx_,_,flag = Person_ReIdentification(img,flag=True)
    else: _,_,flag = Person_ReIdentification(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1

counter = 0       
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_5/1/Sample2*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
    img = Blurring(imgs[0][0])
#     img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Person_ReIdentification(img,flag=True,indx=f'Nassimi')
#         indx_,_,flag = Person_ReIdentification(img,flag=True)
    else: _,_,flag = Person_ReIdentification(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1
        
counter = 0        
for indx,pth in enumerate(glob('./Samples_on_Xception/Sample_5/1/Sample3*')):
    if counter > 16: break
    img = cv2.imread(pth)
    imgs = Person_Detection(img,img,ROI_Region)
    
    img = Blurring(imgs[0][0])
#     img = imgs[0][0]
    
    if indx == 0: indx_,_,flag = Person_ReIdentification(img,flag=True,indx=f'Abdoshah')
#         indx_,_,flag = Person_ReIdentification(img,flag=True)
    else: _,_,flag = Person_ReIdentification(img,flag=True,indx=f'{indx_}')

    if flag: continue
    else: raise 'Error'
    counter += 1

In [20]:
streamer = ThreadedCamera('rtsp:Ashkan321@172.16.60.121:554/Streaming/Channels/001')

In [ ]:
thresh = 7
thresh_on_estimate = 100

old_frame = {}
counter,skipped_frame,skipped_count = 0,0,0

while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    frame = streamer.grab_frame()
        
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        for indx_,(img,cordinations) in enumerate(imgs):
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
#             img_ = Blurring(img)
            img_ = img.copy()

            new_frame[indx_] = {
                'cor':[x+(w/2),y+(h/2)],
                'all_cor':[x,y,w,h],
                'img':img_,
                'label':'Unknown',
                'criterion':'-'
            }
        
        if len(new_frame) == 0: 
            counter += 1
#             writer2.write(img_original)
            
            cv2.imshow('Person_ReIdentification', np.array(img_original))
    
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                cv2.destroyAllWindows()
                streamer.end_frame()
                break
            else: continue
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                if flag_: new_frame[i]['label'] = indx
                else: pass
                new_frame[i]['criterion'] = '*'
        else: pass
        
        
        if len(old_frame) <= len(new_frame):      
            
            for i in old_frame:
                distances_compares,flag_ = [],False
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'Unknown'):
#                 if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[i]['label']):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[i]['label'])]

                if len(elements) == 0:    # agar label ghablan dide nashode boud
                    label = indx if flag_ else old_frame[i]['label']
                    new_frame[j]['label'] = label
                    new_frame[j]['criterion'] = '*'                    
                    labels.append((j,label))

                else:
                    if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'Unknown'): pass #yani p_id ghblan eemal shode
#                     if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[i]['label']): pass
                    
                    elif old_frame[i]['label'] != 'Unknown': # yani p_id emal nashode va label dorost hst
#                     elif not ('Unknown' in old_frame[i]['label']):
                        indx = old_frame[i]['label']
                        flag_ = True
                    
                    else: indx,_,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

                        if len(elements) == 0:
                            new_frame[j]['label'] = indx
                            new_frame[j]['criterion'] = '*'
                            labels.append((j,indx))

                        else:
                            elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
                            if len(elements_) == 0:
                                distances_compares = []
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['criterion'] = '*'
                            
                            else:
                                dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
                                dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))
                                
                                if dist_1 <= dist_2: pass
                                else:
                                    new_frame[j]['label'] = indx
                                    new_frame[j]['criterion'] = '*'
                                    
                                    new_frame[elements[0][0]]['label'] = 'Unknown'
                                    new_frame[elements[0][0]]['criterion'] = '*'
                                    
                                    q = [q for q, v in enumerate(labels) if v[1] == indx][0]
                                    labels[q] = (j,indx)
                    
                    else: pass
        
        else:
            for i in new_frame:
                distances_compares,flag_ = [],False
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'):
#                 if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]

                if len(elements) == 0:
                    new_frame[i]['label'] = indx if flag_ else old_frame[j]['label']
                    new_frame[i]['criterion'] = '*'                    
                    labels.append((j,old_frame[j]['label']))

                else:
                    if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'): pass
#                     if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']): pass
                    
                    elif old_frame[j]['label'] != 'Unknown':
#                     elif not ('Unknown' in old_frame[j]['label']):
                        indx = old_frame[j]['label']
                        flag_ = True
                    
                    else: indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

                        if len(elements) == 0:
                            new_frame[i]['label'] = indx
                            new_frame[i]['criterion'] = '*'
                            labels.append((i,indx))

                        else:
                            elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
                            if len(elements_) == 0:
                                distances_compares = []
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['criterion'] = '*'
                            
                            else:
                                dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
                                dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[i]['cor'])))
                                
                                if dist_1 <= dist_2: pass
                                else:
                                    new_frame[i]['label'] = indx
                                    new_frame[i]['criterion'] = '*'
                                    
                                    new_frame[elements[0][0]]['label'] = 'Unknown'
                                    new_frame[elements[0][0]]['criterion'] = '*'
                                    
                                    q = [q for q, v in enumerate(labels) if v[1] == indx][0]
                                    labels[q] = (i,indx)
                    
                    else: pass             
        
        for i in new_frame:
            img_original = Write_On_Image(new_frame[i]['label'] ,new_frame[i]['all_cor'],img_original)
#             if new_frame[i]['label'] == 'Unknown': Person_ReIdentification(new_frame[i]['img'],flag_unknown=True)
#             else: continue
                
#         Merge_Person_ReIdentification(10)
        
        old_frame = new_frame.copy()

#     writer2.write(img_original)
    counter += 1
    cv2.imshow('Person_ReIdentification', np.array(img_original))
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break
# cap.release()
# writer2.release()

In [19]:
thresh = 8.5
thresh_on_estimate = 6

old_frame = {}
counter,skipped_frame,skipped_count = 0,0,0

while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    frame = streamer.grab_frame()
        
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        for indx_,(img,cordinations) in enumerate(imgs):
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
            img_ = Blurring(img)

            new_frame[indx_] = {
                'cor':[x+(w/2),y+(h/2)],
                'all_cor':[x,y,w,h],
                'img':img_,
                'label':'UnKnown',
                'criterion':'-',
                'status':'-'
            }
        
        if len(new_frame) == 0: 
            counter += 1
            cv2.imshow('Person_ReIdentification', np.array(img_original))
    
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                cv2.destroyAllWindows()
                streamer.end_frame()
                break
            else: continue
                
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                if flag_: new_frame[i]['label'] = indx
                else: pass
                new_frame[i]['criterion'] = '*'
        else: pass
        
        
        if len(old_frame) <= len(new_frame):      
            
            for i in old_frame:
                distances_compares,flag_ = [],False
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'UnKnown'):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[i]['label'])]

                if len(elements) == 0:    # agar label ghablan dide nashode boud
                    label = indx if flag_ else old_frame[i]['label']
                    new_frame[j]['label'] = label
                    new_frame[j]['criterion'] = '*'
                    new_frame[j]['status'] = 'P_Id' if flag_ else 'Estimate'                     
                    labels.append((j,label))

                else:
                    if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'UnKnown'): pass #yani p_id ghblan eemal shode
                    elif old_frame[i]['label'] != 'UnKnown': # yani p_id emal nashode va label dorost hst
                        indx = old_frame[i]['label']
                        flag_ = True
                    else: indx,_,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

                        if len(elements) == 0:
                            new_frame[j]['label'] = indx
                            new_frame[j]['status'] = 'Estimate' if indx == old_frame[i]['label'] else 'P_Id'
                            new_frame[j]['criterion'] = '*'
                            labels.append((j,indx))

                        else:
                            elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
                            if len(elements_) == 0:
                                distances_compares = []
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['status'] = 'P_Id'
                                    new_frame[k]['criterion'] = '*'
                            
                            else:
                                dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
                                dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))
                                
                                if dist_1 <= dist_2: pass
                                else:
                                    new_frame[j]['label'] = indx
                                    new_frame[j]['status'] = 'Estimate' if indx == old_frame[i]['label'] else 'P_Id'
                                    new_frame[j]['criterion'] = '*'
                                    
                                    new_frame[elements[0][0]]['label'] = 'Unknown'
                                    new_frame[elements[0][0]]['criterion'] = '*'
                                    
                                    q = [q for q, v in enumerate(labels) if v[1] == indx][0]
                                    labels[q] = (j,indx)
                    
                    else: pass
        
        else:
            for i in new_frame:
                distances_compares,flag_ = [],False
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
                if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'UnKnown'):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]

                if len(elements) == 0:
                    new_frame[i]['label'] = indx if flag_ else old_frame[j]['label']
                    new_frame[i]['criterion'] = '*'
                    new_frame[i]['status'] = 'P_Id' if flag_ else 'Estimate'                     
                    labels.append((j,old_frame[j]['label']))

                else:
                    if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'UnKnown'): pass

                    elif old_frame[i]['label'] != 'UnKnown':
                        indx = old_frame[i]['label']
                        flag_ = True
                    
                    else: indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx]

                        if len(elements) == 0:
                            new_frame[i]['label'] = indx
                            new_frame[i]['status'] = 'Estimate' if indx == old_frame[j]['label'] else 'P_Id'
                            new_frame[i]['criterion'] = '*'
                            labels.append((i,indx))

                        else:
                            elements_ = [elements[k] for k, v in enumerate(elements) if new_frame[v[1]]['status'] == 'P_Id']
                            if len(elements_) == len(elements):
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['status'] = 'P_Id'
                                    new_frame[k]['criterion'] = '*'
                            
                            else: 
                                print(old_frame[i])
                                for x,y in elements:
                                    print(new_frame[x])
                                print(elements) # Trust the Last Frame  
                                print(elements_)
                                print('**********************')
                    else: pass               
        
        for i in new_frame:
            img_original = Write_On_Image(f"{new_frame[i]['label']} - {new_frame[i]['status']}",new_frame[i]['all_cor'],img_original)
        
        old_frame = new_frame.copy()

    counter += 1
    
    cv2.imshow('Person_ReIdentification', np.array(img_original))
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break

2023-02-20 09:03:16.204640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500


In [18]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_1.mp4')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_1.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# counter = 0
# old_frame = {}

# while True:
#     x,y,h,w = 0,0,0,0
#     new_frame = {}
#     ret,frame = cap.read()
        
#     try: img_original = frame.copy()
#     except AttributeError: break

#     imgs = Person_Detection(frame,img_original,ROI_Region)
    
#     if len(imgs) == 0: pass
#     else: 
#         for indx_,(img,cordinations) in enumerate(imgs):
#             x = cordinations[0]
#             y = cordinations[1]
#             h = cordinations[2]
#             w = cordinations[3]
            
#             img_ = Blurring(img)

#             new_frame[indx_] = {
#                     'cor':[x+(w/2),y+(h/2)],
#                     'all_cor':[x,y,w,h],
#                     'img':img_,
#                     'label':'UnKnown',
#                     'criterion':'-'
#             }
        
#         if (counter % 12) == 0:
#             if len(new_frame) == 0: pass
#             else:
#                 for i in new_frame:
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=10)

#                     if flag_: new_frame[i]['label'] = indx
#                     else: pass

#                     img_original = Write_On_Image(new_frame[i]['label'],new_frame[i]['all_cor'],img_original)

#                 counter += 1
#                 writer2.write(img_original)
#                 old_frame = new_frame.copy()

#             continue
#         else: pass
        
#         if len(new_frame) == 0: continue
#         else: pass
        
#         if len(old_frame) == 0:
#             for i in new_frame:
#                 indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=10)
#                 if flag_: new_frame[i]['label'] = indx
#                 else: pass
#                 new_frame[i]['criterion'] = '*'
#         else: pass
        
        
#         if len(old_frame) <= len(new_frame): 
        
#             for i in old_frame:
#                 distances_compares = []
#                 for j in new_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
#                     distances_compares.append(dist_) 

#                 j = np.argmin(distances_compares)
#                 new_frame[j]['label'] = old_frame[i]['label']

#             for i in new_frame:
#                 if new_frame[i]['criterion'] == '*': pass
#                 else: 
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=10)
#                     if flag_: new_frame[i]['label'] = indx
#                     else: pass
#                 img_original = Write_On_Image(new_frame[i]['label'],new_frame[i]['all_cor'],img_original)
                
#         else:
#             for i in new_frame:
#                 distances_compares = []
#                 for j in old_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
#                     distances_compares.append(dist_) 

#                 j = np.argmin(distances_compares)
#                 new_frame[i]['label'] = old_frame[j]['label']


#             for i in new_frame:
#                 img_original = Write_On_Image(new_frame[i]['label'],new_frame[i]['all_cor'],img_original)

#         if len(new_frame) == 0: pass
#         else: old_frame = new_frame.copy()
        
#     writer2.write(img_original)
#     counter += 1
    
# cap.release()
# writer2.release()           

In [19]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_3.mp4')

# counter = 0
# while True:
#     ret,frame = cap.read()
        
#     cv2.imwrite(f'./Samples_on_Xception/Sample_6/4/frames/{counter}.jpg',frame)
#     counter += 1
    
#     if ret: continue
#     else: break
        
# cap.release()

In [20]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_7.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# thresh = 8
# thresh_on_estimate = 6

# old_frame_clean,old_frame_unknown = {}
# counter,skipped_frame,skipped_count = 0,0,0

# while True:
#     x,y,h,w = 0,0,0,0
    
#     ret,frame = cap.read()
#     new_frame,labels = {},[]
          
#     try: img_original = frame.copy()
#     except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

#     imgs = Person_Detection(frame,img_original,ROI_Region)
    
#     if len(imgs) == 0: pass
#     else:
#         for indx_,(img,cordinations) in enumerate(imgs):
#             x = cordinations[0]
#             y = cordinations[1]
#             h = cordinations[2]
#             w = cordinations[3]
            
#             img_ = Blurring(img)

#             new_frame[indx_] = {
#                 'cor':[x+(w/2),y+(h/2)],
#                 'all_cor':[x,y,w,h],
#                 'img':img_,
#                 'label':'UnKnown',
#                 'criterion':'-'
#             }
        
#         if len(new_frame) == 0: continue
#         else: pass
        
# #         if len(old_frame_clean) == 0 and len(old_frame_unknown) == 0:
# #             for i in new_frame:
# #                 indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
# #                 if flag_: new_frame[i]['label'] = indx
# #                 else: pass
# #                 new_frame[i]['criterion'] = '*'
# #         else: pass
        
        
#         if len(old_frame_clean) <= len(new_frame):      
             
#             for i in old_frame_clean:
#                 distances_compares = []

#                 for j in new_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame_clean[i]['cor'])))
#                     distances_compares.append(dist_) 

#                 j = np.argmin(distances_compares)
#                 new_frame[j]['label'] = old_frame_clean[i]['label']
#                 new_frame[j]['criterion'] = '*'
                
#         else:
#             for j in new_frame:
#                 distances_compares = []

#                 for i in old_frame_clean:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame_clean[i]['cor'])))
#                     distances_compares.append(dist_) 

#                 i = np.argmin(distances_compares)
#                 new_frame[j]['label'] = old_frame_clean[i]['label']
#                 new_frame[j]['criterion'] = '*' 
                
                
#         if len(old_frame_unknown) <= len(new_frame):
        
        
#         else:
#             for j in new_frame:
#                 distances_compares = []

#                 for i in old_frame_clean:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame_clean[i]['cor'])))
#                     distances_compares.append(dist_) 

#                 i = np.argmin(distances_compares)
#                 new_frame[j]['label'] = old_frame_clean[i]['label']
#                 new_frame[j]['criterion'] = '*'            
   
#     writer2.write(img_original)
#     counter += 1
    
# cap.release()
# writer2.release()           

In [21]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# cap.release()

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# for i in range(750,820+1,1):
#     img_original = cv2.imread(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2/{i}.jpg')
#     writer2.write(img_original)
# writer2.release() 

In [ ]:
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4')


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_7.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

thresh = 8.5
thresh_on_estimate = 6

old_frame = {}
counter,skipped_frame,skipped_count = 0,0,0

while True:
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    ret,frame = cap.read()
        
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        for indx_,(img,cordinations) in enumerate(imgs):
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
#             img_ = Blurring(img)
            img_ = img.copy()

            new_frame[indx_] = {
                'cor':[x+(w/2),y+(h/2)],
                'all_cor':[x,y,w,h],
                'img':img_,
                'label':'Unknown',
                'criterion':'-'
            }
        
        if len(new_frame) == 0: 
            counter += 1
            writer2.write(img_original)
            continue
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                if flag_: new_frame[i]['label'] = indx
                else: pass
                new_frame[i]['criterion'] = '*'
        else: pass
        
        
        if len(old_frame) <= len(new_frame):      
            
            for i in old_frame:
                distances_compares,flag_ = [],False
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
#                 if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'Unknown'):
                if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[i]['label']):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[i]['label'])]

                if len(elements) == 0:    # agar label ghablan dide nashode boud
                    label = indx if flag_ else old_frame[i]['label']
                    new_frame[j]['label'] = label
                    new_frame[j]['criterion'] = '*'                    
                    labels.append((j,label))

                else:
#                     if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'Unknown'): pass #yani p_id ghblan eemal shode
                    if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[i]['label']): pass
                    
#                     elif old_frame[i]['label'] != 'Unknown': # yani p_id emal nashode va label dorost hst
                    elif not ('Unknown' in old_frame[i]['label']):
                        indx = old_frame[i]['label']
                        flag_ = True
                    
                    else: indx,_,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

                        if len(elements) == 0:
                            new_frame[j]['label'] = indx
                            new_frame[j]['criterion'] = '*'
                            labels.append((j,indx))

                        else:
                            elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
                            if len(elements_) == 0:
                                distances_compares = []
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['status'] = 'P_Id'
                                    new_frame[k]['criterion'] = '*'
                            
                            else:
                                dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
                                dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))
                                
                                if dist_1 <= dist_2: pass
                                else:
                                    new_frame[j]['label'] = indx
                                    new_frame[j]['criterion'] = '*'
                                    
                                    new_frame[elements[0][0]]['label'] = 'Unknown'
                                    new_frame[elements[0][0]]['criterion'] = '*'
                                    
                                    q = [q for q, v in enumerate(labels) if v[1] == indx][0]
                                    labels[q] = (j,indx)
                    
                    else: pass
        
        else:
            for i in new_frame:
                distances_compares,flag_ = [],False
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_) 
  
                j = np.argmin(distances_compares)
            
#                 if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'):
                if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']):
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh) 
                    if flag_: pass
                    else: continue
                else: pass
                
                elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]

                if len(elements) == 0:
                    new_frame[i]['label'] = indx if flag_ else old_frame[j]['label']
                    new_frame[i]['criterion'] = '*'                    
                    labels.append((j,old_frame[j]['label']))

                else:
#                     if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'Unknown'): pass
                    if ((counter % thresh_on_estimate) == 0) and ('Unknown' in old_frame[j]['label']): pass
                    
#                     elif old_frame[j]['label'] != 'Unknown':
                    elif not ('Unknown' in old_frame[j]['label']):
                        indx = old_frame[j]['label']
                        flag_ = True
                    
                    else: indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)

                    if flag_:
                        elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

                        if len(elements) == 0:
                            new_frame[i]['label'] = indx
                            new_frame[i]['criterion'] = '*'
                            labels.append((i,indx))

                        else:
                            elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
                            if len(elements_) == 0:
                                distances_compares = []
                                for k,_ in elements:
                                    _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
                                    if flag_: distances_compares.append(dist)
                                    else: distances_compares.append(9e9)

                                j = np.argmin(distances_compares)

                                for k in range(len(elements)):
                                    if (k == j) and distances_compares[j] != 9e9:
                                        new_frame[k]['label'] = indx
                                        q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
                                        lables[q] = (k,indx)   
                                    else: new_frame[k]['label'] = 'Unknown'

                                    new_frame[k]['status'] = 'P_Id'
                                    new_frame[k]['criterion'] = '*'
                            
                            else:
                                dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
                                dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[i]['cor'])))
                                
                                if dist_1 <= dist_2: pass
                                else:
                                    new_frame[i]['label'] = indx
                                    new_frame[i]['criterion'] = '*'
                                    
                                    new_frame[elements[0][0]]['label'] = 'Unknown'
                                    new_frame[elements[0][0]]['criterion'] = '*'
                                    
                                    q = [q for q, v in enumerate(labels) if v[1] == indx][0]
                                    labels[q] = (i,indx)
                    
                    else: pass             
        
        for i in new_frame:
            img_original = Write_On_Image(new_frame[i]['label'] ,new_frame[i]['all_cor'],img_original)
            if new_frame[i]['label'] == 'Unknown': Person_ReIdentification(new_frame[i]['img'],flag_unknown=True)
            else: continue
                
        Merge_Person_ReIdentification(10)
        
        old_frame = new_frame.copy()

    writer2.write(img_original)
    counter += 1
    
cap.release()
writer2.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_2.mp4')

counter = 0
while True:
    ret,frame = cap.read()
        
    cv2.imwrite(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2_labled/{counter}.jpg',frame)
    counter += 1
    
    if ret: continue
    else: break
        
cap.release()

In [ ]:
rgb_colors

In [21]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
# # cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_1.mp4')


# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_2.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

# thresh = 8.5
# thresh_on_estimate = 6

# old_frame = {}
# counter,skipped_frame,skipped_count = 0,0,0

# while True:
#     x,y,h,w = 0,0,0,0
    
#     new_frame,labels = {},[]
#     ret,frame = cap.read()
        
#     try: img_original = frame.copy()
#     except AttributeError: break
        
# #     if (skipped_frame % 7) == 0:
# #         if skipped_count < 7:
# #             skipped_count += 1
# # #             writer2.write(img_original)
# #             continue
# #         else:
# #             skipped_count = 0
# #             skipped_frame += 1
# #     else: skipped_frame += 1

#     imgs = Person_Detection(frame,img_original,ROI_Region)
    
#     if len(imgs) == 0: pass
#     else:
#         for indx_,(img,cordinations) in enumerate(imgs):
#             x = cordinations[0]
#             y = cordinations[1]
#             h = cordinations[2]
#             w = cordinations[3]
            
#             img_ = Blurring(img)

#             new_frame[indx_] = {
#                 'cor':[x+(w/2),y+(h/2)],
#                 'all_cor':[x,y,w,h],
#                 'img':img_,
#                 'label':'UnKnown',
#                 'criterion':'-',
#                 'status':'-'
#             }
        
#         if len(new_frame) == 0: 
#             counter += 1
#             writer2.write(img_original)
#             continue
#         else: pass
        
#         if len(old_frame) == 0:
#             for i in new_frame:
#                 indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
#                 if flag_: new_frame[i]['label'] = indx
#                 else: pass
#                 new_frame[i]['criterion'] = '*'
#         else: pass
        
        
#         if len(old_frame) <= len(new_frame):      
            
#             for i in old_frame:
#                 distances_compares,flag_ = [],False
#                 for j in new_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
#                     distances_compares.append(dist_) 
  
#                 j = np.argmin(distances_compares)
            
#                 if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'UnKnown'):
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh) 
#                     if flag_: pass
#                     else: continue
#                 else: pass
                
#                 elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[i]['label'])]

#                 if len(elements) == 0:    # agar label ghablan dide nashode boud
#                     label = indx if flag_ else old_frame[i]['label']
#                     new_frame[j]['label'] = label
#                     new_frame[j]['criterion'] = '*'
#                     new_frame[j]['status'] = 'P_Id' if flag_ else 'Estimate'                     
#                     labels.append((j,label))

#                 else:
#                     if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'UnKnown'): pass #yani p_id ghblan eemal shode
#                     elif old_frame[i]['label'] != 'UnKnown': # yani p_id emal nashode va label dorost hst
#                         indx = old_frame[i]['label']
#                         flag_ = True
#                     else: indx,_,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)

#                     if flag_:
#                         elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx] #dobare check shavd ke elemnts dorost bashad

#                         if len(elements) == 0:
#                             new_frame[j]['label'] = indx
#                             new_frame[j]['status'] = 'Estimate' if indx == old_frame[i]['label'] else 'P_Id'
#                             new_frame[j]['criterion'] = '*'
#                             labels.append((j,indx))

#                         else:
#                             elements_ = [x for x,k in enumerate(old_frame) if old_frame[k]['label'] == indx]
                            
#                             if len(elements_) == 0:
#                                 distances_compares = []
#                                 for k,_ in elements:
#                                     _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
#                                     if flag_: distances_compares.append(dist)
#                                     else: distances_compares.append(9e9)

#                                 j = np.argmin(distances_compares)

#                                 for k in range(len(elements)):
#                                     if (k == j) and distances_compares[j] != 9e9:
#                                         new_frame[k]['label'] = indx
#                                         q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
#                                         lables[q] = (k,indx)   
#                                     else: new_frame[k]['label'] = 'Unknown'

#                                     new_frame[k]['status'] = 'P_Id'
#                                     new_frame[k]['criterion'] = '*'
                            
#                             else:
#                                 dist_1 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[elements[0][0]]['cor'])))
#                                 dist_2 = abs(np.linalg.norm(np.array(old_frame[elements_[0]]['cor'])-np.array(new_frame[j]['cor'])))
                                
#                                 if dist_1 <= dist_2: pass
#                                 else:
#                                     new_frame[j]['label'] = indx
#                                     new_frame[j]['status'] = 'Estimate' if indx == old_frame[i]['label'] else 'P_Id'
#                                     new_frame[j]['criterion'] = '*'
                                    
#                                     new_frame[elements[0][0]]['label'] = 'Unknown'
#                                     new_frame[elements[0][0]]['criterion'] = '*'
                                    
#                                     q = [q for q, v in enumerate(labels) if v[1] == indx][0]
#                                     labels[q] = (j,indx)
                    
#                     else: pass
        
#         else:
#             for i in new_frame:
#                 distances_compares,flag_ = [],False
#                 for j in old_frame:
#                     dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
#                     distances_compares.append(dist_) 
  
#                 j = np.argmin(distances_compares)
            
#                 if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'UnKnown'):
#                     indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh) 
#                     if flag_: pass
#                     else: continue
#                 else: pass
                
#                 elements = [labels[k] for k, v in enumerate(labels) if v[1] == (indx if flag_ else old_frame[j]['label'])]

#                 if len(elements) == 0:
#                     new_frame[i]['label'] = indx if flag_ else old_frame[j]['label']
#                     new_frame[i]['criterion'] = '*'
#                     new_frame[i]['status'] = 'P_Id' if flag_ else 'Estimate'                     
#                     labels.append((j,old_frame[j]['label']))

#                 else:
#                     if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'UnKnown'): pass

#                     elif old_frame[i]['label'] != 'UnKnown':
#                         indx = old_frame[i]['label']
#                         flag_ = True
                    
#                     else: indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)

#                     if flag_:
#                         elements = [labels[k] for k, v in enumerate(labels) if v[1] == indx]

#                         if len(elements) == 0:
#                             new_frame[i]['label'] = indx
#                             new_frame[i]['status'] = 'Estimate' if indx == old_frame[j]['label'] else 'P_Id'
#                             new_frame[i]['criterion'] = '*'
#                             labels.append((i,indx))

#                         else:
#                             elements_ = [elements[k] for k, v in enumerate(elements) if new_frame[v[1]]['status'] == 'P_Id']
#                             if len(elements_) == len(elements):
#                                 for k,_ in elements:
#                                     _,dist,flag_ = Person_ReIdentification(new_frame[k]['img'],thresh=thresh)
#                                     if flag_: distances_compares.append(dist)
#                                     else: distances_compares.append(9e9)

#                                 j = np.argmin(distances_compares)

#                                 for k in range(len(elements)):
#                                     if (k == j) and distances_compares[j] != 9e9:
#                                         new_frame[k]['label'] = indx
#                                         q = [q for q, v in enumerate(lables) if v[1]['label'] == indx][0]
#                                         lables[q] = (k,indx)   
#                                     else: new_frame[k]['label'] = 'Unknown'

#                                     new_frame[k]['status'] = 'P_Id'
#                                     new_frame[k]['criterion'] = '*'
                            
#                             else: 
#                                 print(old_frame[i])
#                                 for x,y in elements:
#                                     print(new_frame[x])
#                                 print(elements) # Trust the Last Frame  
#                                 print(elements_)
#                                 print('**********************')
#                     else: pass               
        
#         for i in new_frame:
#             img_original = Write_On_Image(f"{new_frame[i]['label']} - {new_frame[i]['status']}",new_frame[i]['all_cor'],img_original)
        
#         old_frame = new_frame.copy()

#     writer2.write(img_original)
#     counter += 1
    
# cap.release()
# writer2.release()           

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


{'cor': [1262.0, 547.5], 'all_cor': [790, 203, 944, 689], 'img': array([[[ 62,  52,  40],
        [ 62,  52,  40],
        [ 62,  52,  40],
        ...,
        [ 38,  43,  45],
        [ 38,  43,  45],
        [ 37,  43,  45]],

       [[ 62,  52,  40],
        [ 62,  52,  40],
        [ 62,  52,  40],
        ...,
        [ 38,  43,  45],
        [ 38,  43,  45],
        [ 37,  43,  45]],

       [[ 62,  52,  40],
        [ 62,  52,  40],
        [ 62,  52,  40],
        ...,
        [ 38,  43,  45],
        [ 38,  43,  45],
        [ 38,  43,  45]],

       ...,

       [[244, 250, 248],
        [244, 250, 248],
        [244, 250, 248],
        ...,
        [ 70,  80,  86],
        [ 70,  80,  86],
        [ 70,  80,  86]],

       [[244, 250, 249],
        [244, 250, 249],
        [244, 250, 249],
        ...,
        [ 70,  80,  86],
        [ 70,  80,  86],
        [ 70,  80,  86]],

       [[245, 251, 249],
        [245, 251, 249],
        [245, 251, 249],
        ...,
        [

KeyError: 13

In [22]:
labels

[(0, 13), (1, 'Unknown')]

In [29]:
j

2

In [30]:
new_frame[2]

{'cor': [1575.0, 507.0],
 'all_cor': [1000, 200, 1150, 614],
 'img': array([[[ 86, 104, 122],
         [ 86, 104, 122],
         [ 85, 103, 121],
         ...,
         [154, 184, 206],
         [153, 184, 206],
         [153, 184, 206]],
 
        [[ 86, 104, 122],
         [ 86, 104, 122],
         [ 85, 103, 121],
         ...,
         [154, 184, 206],
         [153, 184, 206],
         [153, 184, 206]],
 
        [[ 85, 104, 122],
         [ 85, 104, 122],
         [ 85, 103, 121],
         ...,
         [154, 184, 206],
         [154, 184, 206],
         [153, 184, 206]],
 
        ...,
 
        [[192, 219, 229],
         [192, 219, 229],
         [192, 219, 229],
         ...,
         [158, 186, 209],
         [162, 191, 214],
         [165, 194, 217]],
 
        [[192, 219, 229],
         [192, 219, 229],
         [192, 219, 229],
         ...,
         [158, 186, 209],
         [162, 191, 214],
         [164, 193, 217]],
 
        [[193, 220, 229],
         [193, 220, 229],


In [ ]:
# cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_2.mp4')
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_test_on_2_Labled_2_1.mp4')

counter = 0
while True:
    ret,frame = cap.read()
        
    cv2.imwrite(f'./Samples_on_Xception/Sample_6/4/frames_test_on_2_labled/{counter}.jpg',frame)
    counter += 1
    
    if ret: continue
    else: break
        
cap.release()

In [23]:
labels

[]

In [19]:
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/video_1.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer2 = cv2.VideoWriter('./Samples_on_Xception/Sample_6/4/video_99.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width,height))

thresh = 8
thresh_on_estimate = 6
time_start,end = 0,0
old_frame = {}
counter,skipped_frame,skipped_count = 0,0,0


while True:
  
    x,y,h,w = 0,0,0,0
    
    new_frame,labels = {},[]
    ret,frame = cap.read()
    
    start = time()  
    
    try: img_original = frame.copy()
    except AttributeError: break
        
#     if (skipped_frame % 7) == 0:
#         if skipped_count < 7:
#             skipped_count += 1
# #             writer2.write(img_original)
#             continue
#         else:
#             skipped_count = 0
#             skipped_frame += 1
#     else: skipped_frame += 1

    imgs = Person_Detection(frame,img_original,ROI_Region)
    
    if len(imgs) == 0: pass
    else:
        for indx_,(img,cordinations) in enumerate(imgs):
            x = cordinations[0]
            y = cordinations[1]
            h = cordinations[2]
            w = cordinations[3]
            
            img_ = Blurring(img)

            new_frame[indx_] = {
                'cor':[x+(w/2),y+(h/2)],
                'all_cor':[x,y,w,h],
                'img':img_,
                'label':'UnKnown',
                'criterion':'-'
            }
        
        if len(new_frame) == 0: continue
        else: pass
        
        if len(old_frame) == 0:
            for i in new_frame:
                indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                if flag_: new_frame[i]['label'] = indx
                else: pass
                new_frame[i]['criterion'] = '*'
        else: pass
        
        
        if len(old_frame) <= len(new_frame): 
        
            for i in old_frame:
                distances_compares = []
                
                for j in new_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[j]['cor'])-np.array(old_frame[i]['cor'])))
                    distances_compares.append(dist_) 
                
                j = np.argmin(distances_compares)

                if ((counter % thresh_on_estimate) == 0) and (old_frame[i]['label'] == 'UnKnown'):
                        indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)
                        if flag_ and not(indx in labels):
                            labels.append(indx)
                            new_frame[j]['label'] = indx
                            new_frame[j]['criterion'] == '*'
                        else: pass
                    

                else:
                    if old_frame[i]['label'] in labels:
                        indx,dist,flag_ = Person_ReIdentification(new_frame[j]['img'],thresh=thresh)
                        if flag_: 
                            if old_frame[i]['label'] == indx: print(f"hi - {counter}")                              
                            labels.append(indx)
                            new_frame[j]['label'] = indx
                            new_frame[j]['criterion'] == '*'  
                        else: pass
                    
                    else:
                        new_frame[j]['criterion'] == '*'
                        labels.append(old_frame[i]['label'])
                        new_frame[j]['label'] = old_frame[i]['label']
        
                
            for i in new_frame:
                if new_frame[i]['criterion'] == '*': pass
                else: 
                    indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                    if flag_: 
                        new_frame[i]['label'] = indx
                        new_frame[i]['criterion'] == '*'
                    else: pass
                img_original = Write_On_Image(new_frame[i]['label'],new_frame[i]['all_cor'],img_original)
                
        else:
            for i in new_frame:
                distances_compares = []
                for j in old_frame:
                    dist_ = abs(np.linalg.norm(np.array(new_frame[i]['cor'])-np.array(old_frame[j]['cor'])))
                    distances_compares.append(dist_)

                j = np.argmin(distances_compares)
                
                if ((counter % thresh_on_estimate) == 0) and (old_frame[j]['label'] == 'UnKnown'):
                        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                        if flag_ and not(indx in labels):
                            labels.append(indx)
                            new_frame[i]['label'] = indx
                        else: pass
                    
                else:
                    if old_frame[j]['label'] in labels:
                        indx,dist,flag_ = Person_ReIdentification(new_frame[i]['img'],thresh=thresh)
                        if flag_: 
                            if old_frame[j]['label'] == indx: print(f"hi - {counter}")
                            labels.append(indx)
                            new_frame[i]['label'] = indx
                        else: pass

                    else:
                        labels.append(old_frame[j]['label'])
                        new_frame[i]['label'] = old_frame[j]['label']
                        

            for i in new_frame:
                img_original = Write_On_Image(new_frame[i]['label'],new_frame[i]['all_cor'],img_original)

        if len(new_frame) == 0: pass
        else: old_frame = new_frame.copy()
        
    writer2.write(img_original)
    end = time()
    
    with open ('./myfile.txt','a') as f:
        f.write(f'{counter} , {end-start} \n')
        
    counter += 1
    
cap.release()
writer2.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


hi - 421
hi - 424
hi - 425
hi - 565
hi - 566
hi - 567
hi - 568
hi - 569


In [20]:
(end - time_start)

104.9048273563385

In [19]:
cap = cv2.VideoCapture('./Samples_on_Xception/Sample_6/4/video_7.mp4')

counter = 0
while True:
    ret,frame = cap.read()
        
    cv2.imwrite(f'./Samples_on_Xception/Sample_6/4/frames_7/{counter}.jpg',frame)
    counter += 1
    
    if ret: continue
    else: break
        
cap.release()

-

-

-

# Evaluate On Cam

In [ ]:
streamer1 = ThreadedCamera('rtsp:Ashkan321@172.16.66.121:554/Streaming/Channels/001')
ROI_Region,ROI_Person = [0,0,0,0],[0,0,0,0]
img1,img_original,imgs,database = None,None,[],[]

In [ ]:
img = cv2.imread('Screenshot from 2022-12-11 09-07-45.png')
imgs = Person_Detection(img,img,ROI_Region)
Person_ReIdentification(imgs[0][0],True)

In [ ]:
plt.imshow(imgs[0][0])

In [ ]:
while True: 
    img_original = streamer1.grab_frame()
    
    try: img_original = Image.fromarray(img_original)
    except AttributeError: continue
    
    if img_original is None: img_original = np.zeros([750,1250,3],dtype=np.uint8)
    else: img_original = cv2.resize(np.array(img_original), (1250, 750))

    if len(database) == 0: pass
    else:
        img1 = img_original.copy()
        try:        
            if ROI_Region[0] == ROI_Region[1] and ROI_Region[2] == ROI_Region[3]:
                if ROI_Region[0] == 0 and ROI_Region[0] == ROI_Region[3]: pass
                else: 
                    img1 = img1[int(ROI_Region[1]):int(ROI_Region[1]+ROI_Region[3]),int(ROI_Region[0]):int(ROI_Region[0]+ROI_Region[2])]
                    img1 = cv2.fastNlMeansDenoisingColored(img1,None,10,20,7,21)
            else: 
                img1 = img1[int(ROI_Region[1]):int(ROI_Region[1]+ROI_Region[3]),int(ROI_Region[0]):int(ROI_Region[0]+ROI_Region[2])]
                img1 = cv2.fastNlMeansDenoisingColored(img1,None,10,20,7,21) 

        except IndexError: pass

        imgs = Person_Detection(img1,img_original,ROI_Region)
    
        if len(imgs) == 0: pass
        else: 
            for img,cordinations in imgs:
                x = cordinations[0]
                y = cordinations[1]
                h = cordinations[2]
                w = cordinations[3]
                
                res,flag = Person_ReIdentification(img)
                if flag:
                    if res: img_original = cv2.rectangle(np.array(img_original), (x+ROI_Region[0], y+ROI_Region[1],abs(w-(x)),abs(h-(y))), (255,0,0), 2)
                    else: continue
                else: continue
                               
    cv2.imshow('Person_ReIdentification', np.array(img_original))
#     cv2.setMouseCallback('Emotion Detection', mouse_click)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer1.end_frame()
        break